<h1>Welcome to the clustering Toronto neighbourhoods project

<h2>Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium
import folium # map rendering library

from bs4 import BeautifulSoup as bs
from IPython.display import display_html

print('Libraries imported.')

Libraries imported.


<h2>Scrape the table of postal codes from the Wikipedia page

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
list = pd.read_html(url)
list1 = list[0]
list1

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park / Ontario Provincial Government,M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned
5,M1HScarborough(Cedarbrae),M2HNorth York(Hillcrest Village),M3HNorth York(Bathurst Manor / Wilson Heights ...,M4HEast York(Thorncliffe Park),M5HDowntown Toronto(Richmond / Adelaide / King),M6HWest Toronto(Dufferin / Dovercourt Village),M7HNot assigned,M8HNot assigned,M9HNot assigned
6,M1JScarborough(Scarborough Village),M2JNorth York(Fairview / Henry Farm / Oriole),M3JNorth York(Northwood Park / York University),M4JEast YorkEast Toronto(The Danforth East),M5JDowntown Toronto(Harbourfront East / Union ...,M6JWest Toronto(Little Portugal / Trinity),M7JNot assigned,M8JNot assigned,M9JNot assigned
7,M1KScarborough(Kennedy Park / Ionview / East B...,M2KNorth York(Bayview Village),M3KNorth York(Downsview)East (CFB Toronto),M4KEast Toronto(The Danforth West / Riverdale),M5KDowntown Toronto(Toronto Dominion Centre / ...,M6KWest Toronto(Brockton / Parkdale Village / ...,M7KNot assigned,M8KNot assigned,M9KNot assigned
8,M1LScarborough(Golden Mile / Clairlea / Oakridge),M2LNorth York(York Mills / Silver Hills),M3LNorth York(Downsview)West,M4LEast Toronto(India Bazaar / The Beaches West),M5LDowntown Toronto(Commerce Court / Victoria ...,M6LNorth York(North Park / Maple Leaf Park / U...,M7LNot assigned,M8LNot assigned,M9LNorth York(Humber Summit)
9,M1MScarborough(Cliffside / Cliffcrest / Scarbo...,M2MNorth York(Willowdale / Newtonbrook),M3MNorth York(Downsview)Central,M4MEast Toronto(Studio District),M5MNorth York(Bedford Park / Lawrence Manor East),M6MYork(Del Ray / Mount Dennis / Keelsdale and...,M7MNot assigned,M8MNot assigned,M9MNorth York(Humberlea / Emery)


<h2> Separate postal codes, boroughs, and neighbourhoods

Assumptions: It appears that the Wikipedia page has been edited to place the postal codes, boroughs, and neighbourhoods in the same table cell without a clear and consistent delimiter. It will therefore be significantly harder to split them as the &lt;br&gt; and &lt;br&#47;&gt; tags are not consistently used. The ( will be used as the delimiter instead, and anything after the ) will be ignored. Neighbourhoods not actually listed in parentheses might not be captured and instead will listed alongside the boroughs.

In [3]:
postalcodes = []
boroughs = []
neighbourhoods = []
for i in range(0,9):
    for j in range (0,20):
        postalcodes.append(list1[i][j][0:3]) #Extracting the postal codes into a list for dataframe
        bornei=(list1[i][j][3:len(list1[i][j])]) #Extracting everything except the postal codes
        bornei2=(bornei.split("(", maxsplit=1)) #Separating boroughs from neighbourhoods
        boroughs.append(bornei2[0]) #Extracting boroughs into a list for dataframe
        if (len(bornei2)==1):
            bornei2.append(bornei) #Duplicating boroughs into neighbourhoods in case none is listed
        bornei3=bornei2[1].split(")", maxsplit=1) #Separating neighbourhoods from extraneous information after
        neigh = bornei3[0].replace (" /",",") #Replacing " /" with ","
        neighbourhoods.append(neigh) #Extracting neighbourhoods into a list for dataframe
data = [postalcodes,boroughs,neighbourhoods]
df = pd.DataFrame.from_records(data) #Converting into dataframe
df2 = df.transpose()
df2.columns=["PostalCodes","Borough","Neighborhood"] #Naming columns
df3 = df2[df2.Borough != 'Not assigned'] #Selecting only postal codes with assigned boroughs
df3.reset_index(drop=True, inplace=True) #resetting index without unassigned postal codes
df3

,PostalCodes,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [4]:
df3.shape

(103, 3)

<h2> Download the geographical coordinates of each postal code

In [5]:
df4 = pd.read_csv("https://cocl.us/Geospatial_data")
df4.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h2>Merge the two dataframes together into a single dataframe by Postal Code

While the data seems to be sorted in order of postal code already, the code below matches the data according to postal codes

In [6]:
df4.rename(columns={'Postal Code':'PostalCodes'},inplace=True)
df5 = pd.merge(df3,df4,on='PostalCodes')
df5

,PostalCodes,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<h2>Include only boroughs that contain "Toronto" in them

In [7]:
df6 = df5[df5['Borough'].str.contains('Toronto',regex=False)]
df6

,PostalCodes,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
40,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


<h2>Create a map of Toronto with neighborhoods superimposed on top.

In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df6['Latitude'], df6['Longitude'], df6['Borough'], df6['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h1>N.B. Occasionally, Github doesn't render Folium maps well; try using https://nbviewer.jupyter.org/ to view it. Alternatively, the direct link can be found <a href="https://nbviewer.jupyter.org/github/seraphchoir/Coursera_Capstone/blob/master/Clustering%20Toronto%20Neighbourhoods%203.ipynb">here</a>.

<h2>Use k-means to cluster neighbourhoods by geographical proximity

In [9]:
kclusters=5
toronto_clustering = df6.drop(['PostalCodes','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = kclusters,random_state=0).fit(toronto_clustering)
kmeans.labels_
df6.insert(0, 'Cluster Labels', kmeans.labels_)
df6

,Cluster Labels,PostalCodes,Borough,Neighborhood,Latitude,Longitude
37,0,M4E,East Toronto,The Beaches,43.676357,-79.293031
40,0,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
41,0,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,0,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,0,M4M,East Toronto,Studio District,43.659526,-79.340923
44,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,2,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,2,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


In [10]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df6['Latitude'], df6['Longitude'], df6['Neighborhood'], df6['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1>N.B. Occasionally, Github doesn't render Folium maps well; try using https://nbviewer.jupyter.org/ to view it. Alternatively, the direct link can be found <a href="https://nbviewer.jupyter.org/github/seraphchoir/Coursera_Capstone/blob/master/Clustering%20Toronto%20Neighbourhoods%203.ipynb">here</a>.

<h2>Observations to be made

It appears that the largest cluster (i.e. with the most neighbourhoods) would be the one in the heart of Downtown Toronto, denoted as Cluster 3. The clusters get smaller with fewer (but probably larger) neighbourhoods as we move away from the city core. 

As the clusters were made from geographical proximity, it would be very easy to see how the clusters were made and how they were formed. The clusters of course have in common their proximity to each other, of which I would posit their relative size and density may be related to their population density.